## Tarea
Va a consistir de 4 pasos basados en lo que aprendimos de como manipular los INEGI shape files.

## Primero
#### Vas a escoger 5 estados y vas a graficar sus vecinos como vimos en el otro archivo. A cada uno de estos grupos los llamaremos grupos de estados.
#### Usa el shapefile de los estados


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
import numpy as np

ModuleNotFoundError: No module named 'geopandas'

In [ ]:

shapefile_path = 'ShapeFilesFiles/Estados/conjunto_de_datos/00ent.shp'
farmacias_path = 'ShapeFilesFiles/conjunto_de_datos/denue_inegi_62_.shp'
estados_seleccionados = ["Puebla", "Sonora", "Michoacan", "Veracruz", "Ciudad de Mexico"]

estados = gpd.read_file(shapefile_path)
farmacias = gpd.read_file(farmacias_path)

In [ ]:
grupos_estados = estados[estados["NOM_ENT"].isin(estados_seleccionados)]

def obtener_vecinos(estado, geodataframe):
    """
    Dado un estado y un GeoDataFrame, encuentra los estados vecinos.
    """
    geometria_estado = geodataframe[geodataframe["NOM_ENT"] == estado].geometry.iloc[0]
    vecinos = geodataframe[geodataframe.geometry.touches(geometria_estado)]
    return vecinos

vecinos_dict = {}
for estado in estados_seleccionados:
    vecinos = obtener_vecinos(estado, estados)
    vecinos_dict[estado] = vecinos

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
estados.plot(ax=ax, color="lightgrey", edgecolor="black")
colors = ["red", "blue", "green", "purple", "orange"]

for i, estado in enumerate(estados_seleccionados):
    grupo = vecinos_dict[estado] 
    grupo.plot(ax=ax, color=colors[i], label=f"Vecinos de {estado}")
    estados[estados["NOM_ENT"] == estado].plot(ax=ax, color="yellow")  

plt.legend()
plt.title("Grupos de Estados y sus Vecinos")
plt.show()


## Segundo
### En cada grupo de estados vas a graficar las farmacias que se encuentran en el área, al igual que un círculo a su alrededor que tenga un radio de igual longitud que la distancia de esa farmacia a la siguiente farmacia más cercana.

In [ ]:
grupos_estados = estados[estados["NOM_ENT"].isin(estados_seleccionados)]

farmacias_gdf = gpd.GeoDataFrame(
    farmacias,
    geometry=gpd.points_from_xy(farmacias.longitud, farmacias.latitud),
    crs="EPSG:4326"  
)

farmacias_en_grupos = farmacias_gdf[farmacias_gdf.geometry.within(grupos_estados.unary_union)]
farmacias_en_grupos = farmacias_en_grupos.to_crs(epsg=3857)
coords = np.array([(geom.x, geom.y) for geom in farmacias_en_grupos.geometry])
tree = cKDTree(coords)
distancias, indices = tree.query(coords, k=2)  
radios = distancias[:, 1]  
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
grupos_estados.plot(ax=ax, color="lightgrey", edgecolor="black")

for i, farmacia in farmacias_en_grupos.iterrows():
    x, y = farmacia.geometry.x, farmacia.geometry.y
    ax.scatter(x, y, color="red", label="Farmacia" if i == 0 else "")
    circle = plt.Circle((x, y), radios[i], color="blue", fill=False, alpha=0.5)
    ax.add_artist(circle)

plt.title("Farmacias y Círculos Basados en la Distancia Más Cercana")
plt.show()


## Tercero
### Por grupo de estados debes calcular cual es la farmacia que en promedio está más alejada de el resto de farmacios.
### Debes graficar las distancias de esta farmacia al resto de las farmacias en el grupo de estados.

In [ ]:
farmacias_por_estado = {}
grupos_estados_geom = grupos_estados.geometry


for estado in estados_seleccionados:
    estado_geom = grupos_estados_geom[grupos_estados["NOM_ENT"] == estado].iloc[0]
    farmacias_en_estado = farmacias_en_grupos[farmacias_en_grupos.geometry.within(estado_geom)]
    farmacias_por_estado[estado] = farmacias_en_estado

for estado, farmacias_en_estado in farmacias_por_estado.items():
    if farmacias_en_estado.empty:
        print(f"No hay farmacias en el grupo de {estado}.")
        continue
    
    
    coords = np.array([(geom.x, geom.y) for geom in farmacias_en_estado.geometry])
    distancia_matrix = cdist(coords, coords)
    distancias_promedio = np.mean(distancia_matrix, axis=1)
    index_max = np.argmax(distancias_promedio)
    farmacia_central = farmacias_en_estado.iloc[index_max]
    distancias_central = distancia_matrix[index_max]
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(distancias_central)), distancias_central, color="blue", alpha=0.7)
    plt.xlabel("Farmacias")
    plt.ylabel("Distancia (metros)")
    plt.title(f"Distancias de la Farmacia con Mayor Promedio en {estado}")
    plt.xticks(range(len(distancias_central)), [f"Farmacia {i}" for i in range(len(distancias_central))], rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(f"En {estado}, la farmacia con mayor promedio de distancia está en las coordenadas: {farmacia_central.geometry.x}, {farmacia_central.geometry.y}.")


## Cuarto
### Calcula cual es el grupo de estados con mayor distancia promedio entre sus farmacias.
### Luego grafica el país completo con el shapefile de estados y grafica el grupo de estados con mayor distancia promedio entre farmacias de un color distinto.

In [ ]:
distancia_promedio_grupo = {}

for estado, farmacias_en_estado in farmacias_por_estado.items():
    if farmacias_en_estado.empty:
        print(f"No hay farmacias en el grupo de {estado}.")
        distancia_promedio_grupo[estado] = 0
        continue
    
    coords = np.array([(geom.x, geom.y) for geom in farmacias_en_estado.geometry])
    distancia_matrix = cdist(coords, coords)
    promedio_distancia = np.mean(distancia_matrix)
    distancia_promedio_grupo[estado] = promedio_distancia

grupo_max_distancia = max(distancia_promedio_grupo, key=distancia_promedio_grupo.get)
print(f"El grupo con mayor distancia promedio es: {grupo_max_distancia} con una distancia promedio de {distancia_promedio_grupo[grupo_max_distancia]} metros.")

grupo_geom = grupos_estados[grupos_estados["NOM_ENT"] == grupo_max_distancia]
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
estados.plot(ax=ax, color="lightgrey", edgecolor="black")
grupo_geom.plot(ax=ax, color="red", edgecolor="black", label=f"Grupo {grupo_max_distancia}")

plt.title("Grupo de Estados con Mayor Distancia Promedio entre Farmacias")
plt.legend()
plt.show()
